<a id='TOC'></a>
# Project: *Analysis of Future of Buffalo Breeds and Milk Production growth in India*

`
By   : Shahapurkar, Gangaprasad
Email: garashah@iu.edu`

<img src="../images/cover_photo.jpg" alt="Cover photo" style="width:=500px;height:=300px;"/>

## Abstract and objective


Water buffalo (**Bubalus bubalis**) is also called *Domestic Water Buffalo* or *Asian Water Buffalo*. It is large bovid originating in Indian subcontinent, Southeast Asia, and China and today found in other regions of world - Europe, Australia, North America, South America and some African countries. There are two extant types recognized based on morphological and behavioural criteria:

1. River Buffalo - Mostly found in Indian subcontinent and further west to the Balkans, Egypt, and Italy
2. Swamp Buffalo - Found from west of Assam through Southeast Asia to the Yangtze valley of China in the east

India is the largest milk producer and consumer compared to other countries in the world, and stands unique in terms of the largest share of milk being produced coming from buffaloes. The aim of this academic project is to study the livestock census data of buffalo breeds in India and their milk production using Empirical Benchmarking analysis method at state level. Looking at the small sample of data, our analysis indicates that we have been seeing increasing trends in past few years in livestock and milk production but there are considerable opportunities to increase production using combined interventions.

## Data files overview 

**1. India Buffalo Data**: Main dataset that contains buffalo breed information of each state in India, Milk production, In-Milk animals, yield per In-Milk animals. Past 6 year data points are represented in data file.

**2. India Demographic Data**: Supplimentary dataset which represents number of villages, districts of each state. Projected population of year 2020 of each state. Agricultural zone category of each state.

## Import libraries

In [ ]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from time import time
import os

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import SelectKBest

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler

from sklearn.decomposition import PCA

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

warnings.simplefilter('ignore')

## Global Configurations

In [ ]:
# Set the color theme for plots
sns.set_style('whitegrid') 
%matplotlib inline 

## Common functions

In [ ]:
'''
Function to load data file in pandas 
dataframe and print shape and information 
of dataframe

Input: Path of file and name of file without extension
Output: Pandas dataframe
'''
def load_data(in_path, name):
    df = pd.read_csv(in_path)
    print(f"{name}: shape is {df.shape}")
    print("\n")
    print(df.info())
    print("\n")
    display(df.head(5))
    return df

In [ ]:
!python --version #check the version of python installed 

In [ ]:
!pip install tabulate # need to install tabulate for printing tables in markdown format

## Import Data

In [ ]:
DATA_DIR = "../data/"               # set data file directory  
buff_data_file_name = "india_buffalo_data" # Main Buffalo data file name
demo_data_file_name = "india_demographics_data" # India demographic data file name

In [ ]:
# Display list of files in data directory
!ls -l $DATA_DIR 

### Main Buffalo Dataset 
This dataset contains 50 features and 36 rows. Each row represents data for one state of country. Notice that we see all the columns of datatype Integer or Float and only column is character which is state name 

In [ ]:
# Load the main data set which contains buffalo information
buffalo_df = load_data(os.path.join(DATA_DIR, f'{buff_data_file_name}.csv'), buff_data_file_name)

### Secondary Dataset - Demographics data
This dataset has 20 features and 36 rows. Each row represents data for one state of country. All the columns of this dataset are Integer and one column is character which is state name

In [ ]:
# Load the secondary data set which contains demographics information
demographic_df = load_data(os.path.join(DATA_DIR, f'{demo_data_file_name}.csv'), demo_data_file_name)

<a id=EDA></a>
## EDA
[Return to start](#TOC)

The goal of this section is to get familiar with the data that will be used for the end to end pipeline. It is very important to explore the data and summarize its main characteristics before diving in the machine learning models. It is also interesting to see how the different features are correlated with the target feature

In [ ]:
pd.set_option('display.float_format', lambda x: f'{x:,.2f}') # Set rounding option for float numbers

In [ ]:
buffalo_df.describe() # Fetch statistical information of buffalo dataset 

In [ ]:
demographic_df.describe() #Fetch statistical information about demographic dataset

### Identify missing values

In [ ]:
# Check top 10 attributes with missing value count in buffalo dataset
percent = (buffalo_df.isnull().sum()/buffalo_df.isnull().count()*100).sort_values(ascending = False).round(2)
sum_missing = buffalo_df.isna().sum().sort_values(ascending = False)

buffalo_missing_data  = pd.concat([percent, sum_missing], axis=1, keys=["Missing %", "Missing Count"])
output = buffalo_missing_data.head(10)

# print dataframe output in markdown format 
print(output.to_markdown(headers=["Feature","Missing %","Missing Count"], tablefmt="grid")) 

In [ ]:
# Check top 10 attributes with missing value count in demographic dataset
percent = (demographic_df.isnull().sum()/demographic_df.isnull().count()*100).sort_values(ascending = False).round(2)
sum_missing = demographic_df.isna().sum().sort_values(ascending = False)

demographic_missing_data  = pd.concat([percent, sum_missing], axis=1, keys=["Missing %", "Missing Count"])
output = demographic_missing_data.head(10)

# print dataframe output in markdown format
print(output.to_markdown(headers=["Feature","Missing %","Missing Count"], tablefmt="grid"))  

### Fill missing values

There are some missing values in main buffalo dataset but there are no mising values in the demograhic dataset. Since all the columns with missing vlaues are number columns we choose to fill them with 0 instead of mean or median method to avoid results.  0 would indicated data was not recorded for this particular attribute. Let us leave the secondary dataset. No action needed on it. 

In [ ]:
buffalo_df.fillna(value=0, inplace=True) # Fill 0 value inplace in buffalo dataset 

### Rest Index

Both the dataset has one record for each state. As state name is unique column in both the dataset so we set sate name as index column and drop it from the column list

In [ ]:
buffalo_df.set_index(["State_name"], drop=True, inplace=True)
demographic_df.set_index(["State_name"], drop=True, inplace=True)

### Data Distribution

Focus of this section would be to identify buffalo distribution in country. We  by buffalo count 

In [ ]:
top_df = buffalo_df[["total_buffalo","total_male","total_female"]] # select only subset of data 
top_df.reset_index(inplace=True)
top_df.columns = ["State","Total","Male","Female"] # rename columns to readable format
top_df.sort_values(by="Total", axis=0, ascending=False, inplace=True) #sort data to select top 10 records
top_df = top_df.iloc[:10]
top_df = top_df[["State","Male","Female"]]
print(top_df.to_markdown(tablefmt="grid")) # print dataframe output in markdown format

In [ ]:
top_transformed = top_df.reset_index().melt(['State', 'index'])
top_transformed.columns = ["State","index","Legend","value"] # rename columns to readable format
output = top_transformed.head()
print(output.to_markdown(tablefmt="grid")) # print dataframe output in markdown format

In [ ]:
sns.barplot(top_transformed.value / 1000000 ,y="State", hue="Legend", data=top_transformed, )
plt.title('Top 10 state by buffalo count', fontsize=16)
plt.xlabel('Buffalo count (Million)', fontsize=12)
plt.ylabel('State', fontsize=12)
plt.yticks(fontsize=12) 
plt.xticks(fontsize=12) 
plt.show()

In [ ]:
labels = ['Female Buffalo', 'Male Buffalo']
sizes = [np.sum(buffalo_df.total_female), np.sum(buffalo_df.total_male)]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',startangle=90)
ax1.axis('equal')
plt.title('Male v/s Female Buffalo Ratio', fontsize=14)
plt.show()

<a id=FeatureEng></a>
## Feature Engineering 
[Return to start](#TOC)

When conducting an end to end Machine Learning project, after exploring and preprocessing the data it is essential to think of feature engineering. It consists of creating new feature(s) based on the features that already exist in the dataset that can be useful for training the model.

### New Features

Calculate total number of buffalos under each category

In [ ]:
buffalo_df["Banni"] = buffalo_df["Banni_male_total"] + buffalo_df["Banni_female_total"]
buffalo_df["Bhadawari"] = buffalo_df["Bhadawari_male_total"] + buffalo_df["Bhadawari_female_total"]
buffalo_df["Chilika"] = buffalo_df["Chilika_male_total"] + buffalo_df["Chilika_female_total"]
buffalo_df["Jaffarabadi"] = buffalo_df["Jaffarabadi_male_total"] + buffalo_df["Jaffarabadi_female_total"]
buffalo_df["Kalahandi"] = buffalo_df["Kalahandi_male_total"] + buffalo_df["Kalahandi_female_total"]
buffalo_df["Marathwadi"] = buffalo_df["Marathwadi_male_total"] + buffalo_df["Marathwadi_female_total"]
buffalo_df["Mehsana"] = buffalo_df["Mehsana_male_total"] + buffalo_df["Mehsana_female_total"]
buffalo_df["Murrah"] = buffalo_df["Murrah_male_total"] + buffalo_df["Murrah_female_total"]
buffalo_df["Nagpuri"] = buffalo_df["Nagpuri_male_total"] + buffalo_df["Nagpuri_female_total"]
buffalo_df["Nili Ravi"] = buffalo_df["Nili_Ravi_male_total"] + buffalo_df["Nili_Ravi_female_total"]
buffalo_df["Non Descript"] = buffalo_df["Non_descript_male_total"] + buffalo_df["Non_descript_female_total"]
buffalo_df["Pandharpuri"] = buffalo_df["Pandharpuri_male_total"] + buffalo_df["Pandharpuri_female_total"]
buffalo_df["Surti"] = buffalo_df["Surti_male_total"] + buffalo_df["Surti_female_total"]
buffalo_df["Toda"] = buffalo_df["Toda_male_total"] + buffalo_df["Toda_female_total"]

In [ ]:
count_df = buffalo_df[["total_buffalo","Banni","Bhadawari","Chilika","Jaffarabadi","Kalahandi","Marathwadi","Mehsana",
                       "Murrah","Nagpuri","Nili Ravi","Non Descript","Pandharpuri","Surti","Toda","total_male",
                      "total_female"]]
count_df.sort_values(by="total_buffalo", axis=0, ascending=False, inplace=True)
count_df.head()

In [ ]:
output = count_df[["Banni","Bhadawari","Chilika","Jaffarabadi","Kalahandi","Marathwadi","Mehsana",
          "Murrah","Nagpuri","Nili Ravi","Non Descript","Pandharpuri","Surti","Toda"]].sum(axis=0)

# print dataframe output in markdown format
print(output.to_markdown(headers=["Breed","Count"], tablefmt="grid", floatfmt=(".0f"))) 

Calculate average of below fields of all past 6 years data points
- **Average in milk** - Average number of In-Milk animals per state (figures in 000 nos)
- **Average yield per In-Milk animal** - Average yield per In-Milk animals per state (figures in kg/day)
- **Average milk production** - Average milk production per state (figures in 000 tones)

In [ ]:
buffalo_df["avg_in_milk"] = (buffalo_df["201314_in_milk"] + buffalo_df["201415_in_milk"] + buffalo_df["201516_in_milk"] + 
                             buffalo_df["201617_in_milk"] + buffalo_df["201718_in_milk"] + buffalo_df["201819_in_milk"])/6

buffalo_df["avg_yield_in_milk"] = (buffalo_df["201314_yield_in_milk"] + buffalo_df["201415_yield_in_milk"] + buffalo_df["201516_yield_in_milk"] + 
                                   buffalo_df["201617_yield_in_milk"] + buffalo_df["201718_yield_in_milk"] + buffalo_df["201819_yield_in_milk"])/6

buffalo_df["avg_milk_production"] = (buffalo_df["201314_milk_production"] + buffalo_df["201415_milk_production"] + buffalo_df["201516_milk_production"] + 
                                     buffalo_df["201617_milk_production"] + buffalo_df["201718_milk_production"] + buffalo_df["201819_milk_production"])/6

In [ ]:
# Fill 0 value inplace in buffalo dataset after new features calculated 
buffalo_df.fillna(value=0, inplace=True) 

buffalo_df[["avg_in_milk","avg_yield_in_milk","avg_milk_production"]].head()

### Correlation with Total Buffalo count

In [ ]:
cols = ["Banni","Bhadawari","Chilika","Jaffarabadi","Kalahandi","Marathwadi","Mehsana",
        "Murrah","Nagpuri","Nili Ravi","Non Descript","Pandharpuri","Surti","Toda",
        "avg_in_milk","avg_yield_in_milk","avg_milk_production", "total_buffalo"]
corr = buffalo_df[cols].corr()

In [ ]:
print("Most Positive Correlations:")
output = corr['total_buffalo'].sort_values(ascending=False).head(5)
print(output.to_markdown(headers=["Feature","%"], tablefmt="grid", floatfmt=(".2f")))

print("\nMost Negative Correlations:")
output = corr['total_buffalo'].sort_values(ascending=False).tail(5)
print(output.to_markdown(headers=["Feature","%"], tablefmt="grid", floatfmt=(".2f")))

### Correlation Observations

Total Buffalo count is not our target feature but it would be interesting to see how the different features are correlated with this feature. 

- Non Descript, Murrah, Average in-milk, Average milk production are highly positve correlated features with total buffalo count. Though Murrah and Non Descript are the only features are actual buffalo breeds and directly impact to total buffalo count but, average in-milk feature takes into consideration total number of in-milk animals and average milk production would be indirect feature.

- Toda , Chilika and Kalahandi are the most highly negative correlated features with total buffalo breeds.

In [ ]:
attributes = ["total_buffalo","avg_in_milk","avg_milk_production","Murrah","Non Descript"]
scatter_matrix(corr[attributes], figsize=(12, 8));

### Buffalo breed analysis of state _**Uttar Pradesh**_ and _**Punjab**_

- **Uttar Pradesh** state is the top most state in _**milk production**_ and total number of buffalo breeds in India. This state has highest number of Murrah **(known worldwide for high yield)**, Non Descript and Bhadawari buffalo breeds.


- **Punjab** is not top milk producing state stands top in the list of _**Average Yield per In-Milk Animal**_. This is becuase ratio of Murrah buffalo breeds which are known for high yield are more compared to Uttar Pradesh

In [ ]:
milk_df = buffalo_df[["total_buffalo","Murrah","avg_milk_production", "avg_yield_in_milk"]]

In [ ]:
milk_df.sort_values(by="avg_milk_production", axis=0, ascending=False, inplace=True)
output = milk_df.head()
print("Top 5 average milk production state")
print(output.to_markdown(headers=["State","Buffalo Count","Murrah Count","Avg milk production","Avg yield per in-milk"], 
                         tablefmt="grid", floatfmt=(".2f")))

In [ ]:
milk_df.sort_values(by="avg_yield_in_milk", axis=0, ascending=False, inplace=True)
output = milk_df.head()
print("Top 5 average yield in-milk state")
print(output.to_markdown(headers=["State","Buffalo Count","Murrah Count","Avg milk production","Avg yield per in-milk"], 
                         tablefmt="grid", floatfmt=(".2f")))

This brings interesting point to check Top breeds in these two states.

In [ ]:
breed_name_list = ["Banni","Bhadawari","Chilika","Jaffarabadi","Kalahandi","Marathwadi","Mehsana",
              "Murrah","Nagpuri","Nili Ravi","Non Descript","Pandharpuri","Surti","Toda"]

In [ ]:
# Filter data for Uttar Pradesh state and Buffalo breeds
top_state_df = buffalo_df.loc[buffalo_df.index == "UTTAR PRADESH"][breed_name_list]
top_state_df.reset_index(drop=True, inplace=True)

temp_df = top_state_df.reset_index().melt() # Transpose data from row to column
temp_df.columns = ["Breed","Count"] # Change column name for readability 
temp_df.sort_values(by="Count", axis=0, ascending=False, inplace=True) # Sort data by breed count

# Display Top 3 records
output = temp_df[:3]
print(output.to_markdown(tablefmt="grid"))

In [ ]:
# capture labels and values for plotting
label_1 = list(output.Breed.values)
sizes_1 = list(output.Count.values)

In [ ]:
top_state_df = buffalo_df.loc[buffalo_df.index == "PUNJAB"][breed_name_list]
top_state_df.reset_index(drop=True, inplace=True)

temp_df = top_state_df.reset_index().melt() # Transpose data from row to column
temp_df.columns = ["Breed","Count"] # Change column name for readability 
temp_df.sort_values(by="Count", axis=0, ascending=False, inplace=True) # Sort data by breed count

# Display Top 3 records
output = temp_df[:3]
print(output.to_markdown(tablefmt="grid"))

In [ ]:
# capture labels and values for plotting
label_2 = list(output.Breed.values)
sizes_2 = list(output.Count.values)

Let us plot the data of two states side by side to get a visual picture of distribution of buffalo breeds

In [ ]:
fig, (ax0,ax1) = plt.subplots(1, 2)
fig.suptitle('Top 3 Buffalo breeds', fontsize=16)
ax0.set_title('Uttar Pradesh', fontsize=14)
ax1.set_title('Punjab', fontsize=14)
ax0.pie(sizes_1, labels=label_1, autopct='%1.1f%%',startangle=90)
ax1.pie(sizes_2, labels=label_2, autopct='%1.1f%%',startangle=90)
plt.show()

<a id=Modeling></a>
## Modelling
[Return to start](#TOC)

Now that we have explored the data, cleaned it, preprocessed it and added a new feature to it, we can start the modelling part of the project by applying algorithms.

### Merge Dataset

In [ ]:
# Merge main and demograhic dataset
data = buffalo_df.merge(demographic_df, how='left', on='State_name')
data.reset_index(inplace=True)
data.shape

### Define features and labels

Since this dataset is a small and it is not labelled dataset, we would be considering our calculated field **avg milk production** as target variable and do co-variate analysis with demographic features 

In [ ]:
# Define Features and labels
final_data = data.copy()

label = final_data['avg_milk_production']
features = final_data.drop(['State_name'], axis=1)

### Extract numerical and categorical features

This dataset contains all numerical features. Only state name could be called as categorical feature hence we will ignore this field 

In [ ]:
#Setup attributes for pipeline
num_attribs = data.select_dtypes(exclude='object').columns
cat_attribs = data.select_dtypes(include='object').columns

In [ ]:
cat_attribs

In [ ]:
num_attribs

### Normalize Data

Since the feature of the dataset are of different scale we would need to normalize the data before using it further

In [ ]:
dfNumCols = data[num_attribs]

In [ ]:
# Normalize data and create dataframe from the result
num_encode = Normalizer().fit(dfNumCols)

df_num_cols = pd.DataFrame(
    num_encode.transform(dfNumCols), index=dfNumCols.index, 
    columns=list(num_attribs)
)
df_num_cols.describe()

### Empirical Benchmarking Model & Covariance analysis

There are two dominant approach of economic modelling to estimate the production behavior - **Empirical Benchmarking** and **Stochastic Frontier Analysis**. Empirical Benchmarking is simple modelling method, and it is one of the two dominant approach. This method was used to analyze past 6 years of data points available in the livestock dataset. In this approach milk production data of past 6 years was averaged. Top 10 states with most milk production reported were compared with average of the whole sample.

Covariance analysis was performed by with other demographics attributes  

In [ ]:
model = data[["State_name","total_buffalo", "total_male", "total_female",
                       "avg_in_milk","avg_yield_in_milk","avg_milk_production","proj_population_2020"]]
model.sort_values(by="avg_milk_production", axis=0, ascending=False, inplace=True)
model_top10 = model[["State_name","avg_milk_production","proj_population_2020"]].head(10)

In [ ]:
mean_avg_milk_production_sample = np.round(data.avg_milk_production.mean(), decimals=2)
print("Average Milk Production from Buffalos for whole sample         :", mean_avg_milk_production_sample)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
sns.barplot(x=model_top10.State_name,y=model_top10.avg_milk_production, data=model_top10)
plt.axhline(y=mean_avg_milk_production_sample,linewidth=1)
plt.xticks(rotation=80)
plt.xlabel('State', fontsize=14)
plt.ylabel('Avg Milk Production (tones)', fontsize=14)
plt.title('Avg Milk Production in top 10 states', fontsize=16)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
sns.barplot(x=model_top10.State_name,y=model_top10.proj_population_2020/1000000, data=model_top10)
plt.xticks(rotation=80)
plt.xlabel('State', fontsize=14)
plt.ylabel('Population (Million)', fontsize=14)
plt.title('Projected Population of 2020', fontsize=16)
plt.show()

#### Covariance Analysis

Let us check coorelation with limited number of features for our target variable **average milk production**

In [ ]:
demo_features = ['total_male', 'total_female','avg_in_milk', 'avg_yield_in_milk',
'avg_milk_production', 'district_count', 'village_count',
'official_area_sqkm', 'proj_population_2020', 'agro_climatic_zone1',
'agro_climatic_zone2', 'agro_climatic_zone3', 'agro_climatic_zone4',
'agro_climatic_zone5', 'agro_climatic_zone6', 'agro_climatic_zone7',
'agro_climatic_zone8', 'agro_climatic_zone9', 'agro_climatic_zone10',
'agro_climatic_zone11', 'agro_climatic_zone12', 'agro_climatic_zone13',
'agro_climatic_zone14', 'agro_climatic_zone15']

In [ ]:
corr = df_num_cols[demo_features].corr()

print("Most Positive Correlations:")
output = corr['avg_milk_production'].sort_values(ascending=False).head(5)
print(output.to_markdown(headers=["Feature","%"], tablefmt="grid", floatfmt=(".2f")))

print("\nMost Negative Correlations:")
output = corr['avg_milk_production'].sort_values(ascending=False).tail(5)
print(output.to_markdown(headers=["Feature","%"], tablefmt="grid", floatfmt=(".2f")))

Now let us check it at boarder level with all our calculated fields 

In [ ]:
# Calculate coorelation
corr1_df =  data.corr()

# Delete non relevant fields
delete_fields = ['Banni_male_total', 'Banni_female_total', 'Bhadawari_male_total','Bhadawari_female_total', 
                 'Chilika_male_total', 'Chilika_female_total','Jaffarabadi_male_total', 'Jaffarabadi_female_total', 
                 'Kalahandi_male_total', 'Kalahandi_female_total', 'Marathwadi_male_total', 'Marathwadi_female_total', 
                 'Mehsana_male_total', 'Mehsana_female_total', 'Murrah_male_total', 'Murrah_female_total', 
                 'Nagpuri_male_total', 'Nagpuri_female_total', 'Nili_Ravi_male_total', 'Nili_Ravi_female_total', 
                 'Non_descript_male_total', 'Non_descript_female_total', 'Pandharpuri_male_total', 
                 'Pandharpuri_female_total', 'Surti_male_total', 'Surti_female_total', 'Toda_male_total', 
                 'Toda_female_total', 'total_buffalo', '201314_in_milk', '201415_in_milk', '201516_in_milk', 
                 '201617_in_milk', '201718_in_milk', '201819_in_milk', '201314_yield_in_milk', '201415_yield_in_milk', 
                 '201516_yield_in_milk', '201617_yield_in_milk', '201718_yield_in_milk', '201819_yield_in_milk', 
                 '201314_milk_production', '201415_milk_production', '201516_milk_production', '201617_milk_production', 
                 '201718_milk_production', '201819_milk_production','agro_climatic_zone1', 'agro_climatic_zone2', 
                 'agro_climatic_zone3', 'agro_climatic_zone4', 'agro_climatic_zone5', 'agro_climatic_zone6', 
                 'agro_climatic_zone7', 'agro_climatic_zone8', 'agro_climatic_zone9', 'agro_climatic_zone10', 
                 'agro_climatic_zone11', 'agro_climatic_zone12', 'agro_climatic_zone13', 'agro_climatic_zone14', 
                 'agro_climatic_zone15']

# drop rows
corr1_df.drop(delete_fields, inplace=True)

# drop columns
corr1_df.drop(delete_fields, axis=1, inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10))
# mask
mask = np.triu(np.ones_like(corr1_df, dtype=np.bool))
# adjust mask and df
mask = mask[1:, :-1]
corr = corr1_df.iloc[1:,:-1].copy()
# plot heatmap
sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap='Blues',
           vmin=-1, vmax=1, cbar_kws={"shrink": .8})
# yticks
plt.title("Covriance Map", fontsize=16)
plt.yticks(rotation=0)
plt.show()

### Linear Regression Model

#### Custom Transformers

In [ ]:
# Custom transformer to select columns from dataframe
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

#### Define Pipeline

In [ ]:
# Define pipeline
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)), 
        ('std_scaler', Normalizer())
    ])

In [ ]:
# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
pca = PCA()

# Linear Regression without parameters
linear = LinearRegression()

full_pipeline_with_predictor = Pipeline([
        ("preparation", num_pipeline),
        ("pca",pca),
        ("linear", linear)
    ])

# Parameters of pipelines:
param_grid = {
    'pca__n_components': [5, 15, 30, 45, 64]
}

In [ ]:
# Apply GridSearch with predictors 
search = GridSearchCV(full_pipeline_with_predictor,param_grid, n_jobs=-1)
search.fit(features, label)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

# Plot the PCA spectrum
pca.fit(features)

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(6, 6))
ax0.plot(np.arange(1, pca.n_components_ + 1), pca.explained_variance_ratio_, '+', linewidth=2)
ax0.set_ylabel('PCA explained variance ratio')

ax0.axvline(search.best_estimator_.named_steps['pca'].n_components, linestyle=':', label='n_components chosen')
ax0.legend(prop=dict(size=12))

# For each number of components, find the best classifier results
results = pd.DataFrame(search.cv_results_)

components_col = 'param_pca__n_components'
best_clfs = results.groupby(components_col).apply(lambda g: g.nlargest(1, 'mean_test_score'))
best_clfs.plot(x=components_col, y='mean_test_score', yerr='std_test_score',legend=False, ax=ax1)
ax1.set_ylabel('Classification accuracy (val)')
ax1.set_xlabel('n_components')

plt.xlim(-1, 70)

plt.tight_layout()
plt.show()

In [ ]:
results

<a id=Evaluate></a>
## Evaluation, reporting, analysis
[Return to start](#TOC)

**Empirical Benchmarking model** denotes that top 10 states are performing above average. There is possibility to increase the yield then the current attainable yield, evaluating the other socio-economic factors like agricultural data, population, climate factors etc. 

Correlation analysis done with other demographic features like population and climatic zones suggest that average growth is highly correlated with average in-milk animals, female buffaloes and climatic zone region which comes under **Zone 6: Trans-Gangetic Plains**.This factors would be highly contributing to average milk production growth

Due to absence of sufficient granular data analysis had to be done on state level rollup data. The sample linear regression modle attempted above did not yield satisfactory and meaningfull results. Based on the data trends it appears that it is possible to increase the production past current attenable numbers. However, this would need to combine different methods and multiple strategies.